In [3]:
import os
import pandas as pd

# Locating the Data
First things first: let's check where our results are located. Use this script in combination with your command to see where the results should be if your naming matches the example config file.

In [4]:
command_str = "python cli.py 'seed=range(0,10)' method=1,3,7"
seeds = range(0,10)
methods = [1,3,7]
benchmarks = [4]
base_path = "results"
experiment_directories = []

for b in benchmarks:
    for m in methods:
        for s in seeds:
            experiment_directories.append(os.path.join(base_path, f"{m}_{b}", f"seed_{s}"))

print(experiment_directories)

['results/1_4/seed_0', 'results/1_4/seed_1', 'results/1_4/seed_2', 'results/1_4/seed_3', 'results/1_4/seed_4', 'results/1_4/seed_5', 'results/1_4/seed_6', 'results/1_4/seed_7', 'results/1_4/seed_8', 'results/1_4/seed_9', 'results/3_4/seed_0', 'results/3_4/seed_1', 'results/3_4/seed_2', 'results/3_4/seed_3', 'results/3_4/seed_4', 'results/3_4/seed_5', 'results/3_4/seed_6', 'results/3_4/seed_7', 'results/3_4/seed_8', 'results/3_4/seed_9', 'results/7_4/seed_0', 'results/7_4/seed_1', 'results/7_4/seed_2', 'results/7_4/seed_3', 'results/7_4/seed_4', 'results/7_4/seed_5', 'results/7_4/seed_6', 'results/7_4/seed_7', 'results/7_4/seed_8', 'results/7_4/seed_9']


# Checking if Everything Ran Successfully
Now that we know the location of our data, we can check if it's complete. For this purpose, you need to define a function that takes a directory for a single run and returns a boolean signal if this run is complete.

In [7]:
def job_done(path_str):
    path = os.path.join(path_str, "done.txt")
    if os.path.exists(path):
        with open(path, "r") as f:
            content = f.read()
            if "yes" in content:
                return True
    return False

In [9]:
missing = []
for d in experiment_directories:
    if not job_done(d):
        missing.append(d)

if len(missing) > 0:
    print(f"Done: {len(experiment_directories)-len(missing)}/{len(experiment_directories)}")
    print("Missing jobs:")
    for m in missing:
        print(m)
else:
    print("All jobs done!")

Done: 0/30
Missing jobs:
results/1_4/seed_0
results/1_4/seed_1
results/1_4/seed_2
results/1_4/seed_3
results/1_4/seed_4
results/1_4/seed_5
results/1_4/seed_6
results/1_4/seed_7
results/1_4/seed_8
results/1_4/seed_9
results/3_4/seed_0
results/3_4/seed_1
results/3_4/seed_2
results/3_4/seed_3
results/3_4/seed_4
results/3_4/seed_5
results/3_4/seed_6
results/3_4/seed_7
results/3_4/seed_8
results/3_4/seed_9
results/7_4/seed_0
results/7_4/seed_1
results/7_4/seed_2
results/7_4/seed_3
results/7_4/seed_4
results/7_4/seed_5
results/7_4/seed_6
results/7_4/seed_7
results/7_4/seed_8
results/7_4/seed_9


# Make Runscripts to Rerun Missing Runs
Since it's possible some runs die before finishing, we need to rerun them at times. Here we can generate scripts to do this.

In [12]:
all_files = []
for b in benchmarks:
    for m in methods:
        for s in seeds:
            filepath = f"missing_method_{m}_benchmark_{b}_seed_{s}.sh"
            command = f"python cli.py seed={s} method={m} benchmark={b}"
            first = True
            with open(filepath, "a+") as f:
                if first:
                    first = False
                    slurm_string = f"""#!/bin/bash \n#SBATCH --error={m}_{b}.err \n#SBATCH --job-name=missing \n#SBATCH --mem=10GB \n#SBATCH --output={m}_{b}.out \n#SBATCH --partition=ai,tnt \n#SBATCH --time=1440 \nconda activate my_env"""
                    f.write(slurm_string)
                    f.write("\n")
                f.write(command)
                f.write("\n")
            all_files.append(filepath)

with open("submit_all_missing.sh", "a+") as f:
    for file in all_files:
        f.write(f"sbatch {file}")
        f.write("\n")

# now we could 'sbatch submit_all_missing.sh' in the terminal to run the missing jobs

# Loading the Data
There are multiple ways to then work with your data. We'll look at getting all of it into a single DataFrame here - if you need to load things differently, you'll have to edit the code. You can either use these dataframes directly for plotting or, even better, save one .csv per experiment. This will make rerunning and data loading much easier!

In [ ]:
dfs = []
for d in experiment_directories:
    path = os.path.join(d, "performance.csv")
    df = pd.read_csv(path)

    # Add sweep parameters
    seed = d.split("_")[-1][-1]
    df["seed"] = [seed] * len(df)
    method = d.split("_")[-2]
    df["method"] = [method] * len(df)

    dfs.append(df)

data = pd.concat(dfs)
print(data)

In [ ]:
# save to single csv file
data.to_csv("all_performances.csv")

# split into separate files by method
for m in df["method"].unique():
    df[df["method"] == m].to_csv(f"{m}_performances.csv")